In [ ]:
def dm_test(timeseries=None, power=2, **model_preds):
    """
    Perform Diebold-Mariano test for predictive accuracy comparison.

    Parameters:
    - timeseries (DataFrame): Actual time series data.
    - power (int): Power for residuals calculation.
    - **model_preds: Dictionary of model predictions.

    Returns:
    - DataFrame: Results of the Diebold-Mariano test including t-values and p-values.
    """

    import pandas as pd
    from itertools import combinations
    from statsmodels.api import OLS

    # Combine model predictions into a single DataFrame
    predictions_df = pd.concat([pd.DataFrame(pred).reset_index(drop=True) for pred in model_preds.values()], axis=1)
    predictions_df.columns = model_preds.keys()

    # Prepare the actual time series data
    timeseries_df = pd.DataFrame(timeseries).reset_index(drop=True)

    # Calculate squared residuals
    residuals_matrix_sq = (timeseries_df.to_numpy() - predictions_df) ** power

    # Generate combinations of model pairs for comparison
    residuals_matrix_combinations = combinations(residuals_matrix_sq.columns, 2)
    possible_combinations = [f"(e_{col1})^2-(e_{col2})^2" for col1, col2 in residuals_matrix_combinations]

    # Calculate t-values and p-values for each pair of models
    t_value_list = []
    p_value_list = []
    for pair in combinations(residuals_matrix_sq.columns, 2):
        col1, col2 = pair
        d = residuals_matrix_sq[col1] - residuals_matrix_sq[col2]
        model = OLS(endog=d, exog=pd.DataFrame({"c0": [1] * len(d)})).fit()
        p_value_list.append(model.t_test("c0=0", use_t=True).pvalue)
        t_value_list.append(model.t_test("c0=0", use_t=True).statistic[0][0])

    # Create output DataFrame with t-values and p-values indexed by model pair combinations
    output = pd.DataFrame({"t_value": t_value_list, "p_value": p_value_list}, index=possible_combinations)

    return output


# Example usage
mm = dm_test(timeseries=inflation, model_1=model_1.predict(), model_2=model_2.predict(), model_0=model_0.predict()) 
# Note: You might just add as many model as you got here in that way. 
mm
